# DT Assignment

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

$$ Gini(D_{i}) = 1 - \sum_{j = 1}^{x} P_{j}^2 $$

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [3]:
def get_gini(df, label):
    p_yes = (df[label]=='yes').mean()
    p_no = 1 - p_yes
    gini = 1 - p_yes**2 - p_no**2    
    return gini

In [4]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    result=[]
    for i in range(1, len(uniques)):
        result += [list(x) for x in combinations(uniques, i)]
    return result

In [6]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
def get_attribute_gini_index(df, attribute, label):
    splits = get_binary_split(df, attribute)
    result = {}
    for split in splits:
        # 카테고리에 해당되는 부분
        pos = df[df[attribute].isin(split)]
        # 아닌 부분
        neg = df[~df[attribute].isin(split)]
        # gini값 계산
        gini = (len(pos) * get_gini(pos, label) + len(neg) * get_gini(neg, label)) / len(df)
        result['_'.join(split)] = gini    
    return result

In [8]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{'high': 0.4428571428571429,
 'medium': 0.4583333333333333,
 'low': 0.45,
 'high_medium': 0.45,
 'high_low': 0.4583333333333333,
 'medium_low': 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [9]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

('high', 0.4428571428571429)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [10]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]

# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    min_gini = min(get_attribute_gini_index(pd_data, feature, label).items())[1]
    print(f'Minimum Gini index of {feature} : {min_gini : .4f}')

Minimum Gini index of age :  0.3571
Minimum Gini index of income :  0.4429
Minimum Gini index of student :  0.3673
Minimum Gini index of credit_rating :  0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [11]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{'youth': 0.39365079365079364,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'youth_middle_aged': 0.4571428571428572,
 'youth_senior': 0.35714285714285715,
 'middle_aged_senior': 0.39365079365079364}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## 위에서 선정한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각 다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [12]:
#선정한 feature로 데이터프레임 split을 해주세요.

df_1 = pd_data[pd_data['age'] == 'middle_aged']
df_2 = pd_data[pd_data['age'] != 'middle_aged']

In [13]:
df_1.head() # split 결과 확인 

,age,income,student,credit_rating,class_buys_computer
2,middle_aged,high,no,fair,yes
6,middle_aged,low,yes,excellent,yes
11,middle_aged,medium,no,excellent,yes
12,middle_aged,high,yes,fair,yes


In [14]:
df_2.head() # split 결과 확인

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [15]:
# 각 변수를 대상으로 반복문 수행
# 결과는 소수점 이하 4자리까지 출력

features = ['income', 'student', 'credit_rating']

for feature in features :
    print('Age : middel_aged,', 'Minimum Gini index of ', feature, ':', round(min(get_attribute_gini_index(df_1, feature, label).values()), 4))
          
print("#############################################################")
          
for feature in features :
    print('Age : youth_senior,', 'Minimum Gini index of ', feature, ':', round(min(get_attribute_gini_index(df_2, feature, label).values()), 4))
          

Age : middel_aged, Minimum Gini index of  income : 0.0
Age : middel_aged, Minimum Gini index of  student : 0.0
Age : middel_aged, Minimum Gini index of  credit_rating : 0.0
#############################################################
Age : youth_senior, Minimum Gini index of  income : 0.375
Age : youth_senior, Minimum Gini index of  student : 0.32
Age : youth_senior, Minimum Gini index of  credit_rating : 0.4167


## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [16]:
from math import log2

def getEntropy(df, feature) :
    
    entropy = 0
    # pandas.Series.value_counts 함수를 이용하여 feature의 class별 확률을 계산
    ps = df[feature].value_counts() / len(df)
    for p in ps:
        # entropy에 p * log2p를 빼주어 최종 entropy를 계산
        entropy -= p * log2(p)
    
    return(entropy)

In [17]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

**Information Gain**

$$ Gain(S, A) = E(S) - I(S, A) $$

$$ I(S, A) = \sum_{i}\frac{|S_i|}{S} E(S_i) $$

$$ Gain(S, A) = E(S) - I(S, A) = E(S) - \sum_{i}\frac{|S_i|}{S} E(S_i) $$

In [18]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
    # 딕셔너리형 변수 result 선언    
    result = {}
    # 목표변수에 대한 Entropy 를 구하기
    info_D = getEntropy(df, feature)
    # 목표변수(feature)를 제외한 나머지 변수들의 변수명을 리스트 형태로 저장
    columns = list(df.loc[:, df.columns != feature])
    
    # Column별로 반복
    for i in columns:
        # 가중치 * Entropy를 저장할 수 있는 변수 pre_info 선언
        pre_info = 0
        for j in list(df[i].unique()):
            # S : 전체 데이터 개수
            S = len(df[feature])
            # S_j : 전체 데이터 중 i 컬럼의 값이 j인 데이터의 개수(=가중치)
            S_j = sum(df[i] == j)
            # pre_info에 가중치 * Entropy 누적합 저장
            pre_info += (S_j / S) * getEntropy(df[df[i] == j], feature)
        # 최종 결과는 목표변수의 Entropy - pre_info
        result[i] = info_D - pre_info
    
    return(result)

In [19]:
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}

Information Gain 값이 가장 큰, 즉, Entropy를 가장 많이 줄인 age 변수를 첫 번째 기준으로 채택함.